# Linear Regression

In [15]:
import pandas as pd

# load data
df_cloud = pd.read_csv("../data/incidentsByDateCountry.csv")
df_strom = pd.read_csv("../storms_pre_processed.csv")
df_strom = df_strom.rename(columns={"Country": "country"})

df_strom["country"].replace(
    to_replace={"United Kingdom of Great Britain and Northern Ireland (the)": "United Kingdom",
                "Korea (the Republic of)": "South Korea",
                "United States of America (the)": "USA",
                "Taiwan (Province of China)": "Taiwan",
                "Netherlands (the)": "Netherlands",
                "Qatar": "Katar"
                }, inplace=True)

df_joined = pd.merge(df_cloud, df_strom, on=["date", "country"], how="inner")

#print(sorted(df_cloud["country"].drop_duplicates().dropna().to_list()))
#print(sorted(df_strom["country"].drop_duplicates().dropna().to_list()))
#print(sorted(df_joined["country"].drop_duplicates().dropna().to_list()))
cloud_set = set(df_cloud["country"].drop_duplicates().dropna().to_list())
joined_set = set(df_joined["country"].drop_duplicates().dropna().to_list())
not_intersection = joined_set ^ cloud_set
print(not_intersection)
print(df_cloud.shape)
print(df_strom.shape)
print(df_joined.shape)

['ASIA', 'Australia', 'Belgium', 'Brazil', 'Canada', 'Chile', 'Europe', 'Finland', 'France', 'Germany', 'Global', 'Hong Kong', 'India', 'Indonesia', 'Israel', 'Italy', 'Japan', 'Katar', 'Netherlands', 'Poland', 'Singapore', 'South Korea', 'Spain', 'Switzerland', 'Taiwan', 'USA', 'United Kingdom']
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Bahamas (the)', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Canary Is', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros (the)', 'Congo (the Democratic Republic of the)', 'Congo (the)', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic (the)', 'Côte d’Ivoire', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic (the)', 'Ecuad

In [ ]:
df_joined.to_csv("../data/full.csv")